In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

In [2]:
path = 'cs5228-2310-final-project/'
df_train = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')

In [3]:
def from_lat_long_to_dis(latA, lonA, latB, lonB):
    ra = 6378140
    rb = 6356755
    flatten = (ra - rb) / ra

    rad_lat_A = np.radians(latA)
    rad_lng_A = np.radians(lonA)
    rad_lat_B = np.radians(latB)
    rad_lng_B = np.radians(lonB)

    pA = np.arctan(rb / ra * np.tan(rad_lat_A))
    pB = np.arctan(rb / ra * np.tan(rad_lat_B))
    x = np.arccos(np.sin(pA) * np.sin(pB) + np.cos(pA) * np.cos(pB) * np.cos(rad_lng_A - rad_lng_B))
    c1 = (np.sin(x) - x) * (np.sin(pA) + np.sin(pB)) ** 2 / np.cos(x / 2) ** 2
    c2 = (np.sin(x) + x) * (np.sin(pA) - np.sin(pB)) ** 2 / np.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance /= 1000 # convert to km
    return distance




In [4]:
from_lat_long_to_dis(26.08, 119.28, 24.26, 118.04)# fuzhou to xiamen

237.2152647407692

In [5]:
df_train['latitude'] *= 2

In [8]:
df_train = df_train[:10000]

In [9]:
##KNN model
from sklearn.neighbors import KNeighborsRegressor

# train
X_train = df_train[['latitude', 'longitude']]
y_train = df_train['monthly_rent']

# fit
knn = KNeighborsRegressor(n_neighbors=100)
knn.fit(X_train, y_train)

# draw fitted surface
# create meshgrid
x_min, x_max = X_train['latitude'].min(), X_train['latitude'].max()
y_min, y_max = X_train['longitude'].min(), X_train['longitude'].max()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

# predict
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)



c:\ProgramData\anaconda3\envs\CS5228\lib\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but KNeighborsRegressor was fitted with feature names



In [10]:
#draw 3D surface
from plotly import graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure(data=[go.Surface(z=Z, x=xx, y=yy)])

fig.update_layout(title='KNN Surface', autosize=False,
                    width=500, height=500,
                    margin=dict(l=65, r=50, b=65, t=90))
fig.show()
